In [2]:
file_path = 'KC_case_data\xa0.csv'

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv(file_path, parse_dates=['date'])

<strong>Данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020</strong>

In [5]:
df.head(10)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
5,2020-01-01,app_start,NaN,android,5148476,male,Saint-Petersburg,-
6,2020-01-01,app_start,NaN,android,8383386,female,Saint-Petersburg,vk_ads
7,2020-01-01,app_start,NaN,android,8581267,female,Saint-Petersburg,google_ads
8,2020-01-01,app_start,NaN,ios,10144701,female,Moscow,-
9,2020-01-01,app_start,NaN,ios,10274354,female,Saint-Petersburg,instagram_ads


<h1>Описание данных</h1>

- date – дата совершения события

- event - событие:

    app_install – установка приложения\
    app_start – открыть приложения\
    register – зарегистрироваться \
    search – перейти на страницу поиска товаров (каталог)\
    open_item – открыть товар\
    choose_item – отправить товар в корзину\
    tap_basket – перейти в корзину\
    purchase – подтверждение покупки\
<>
- purchase_sum – стоимость покупки (при совершении события ‘purchase’)
- os_name – платформа пользователя
- device_id – идентификатор устройства пользователя
- gender – пол пользователя
- city – город пользователя
- urm_source – канал, с которого пришел пользователь:

    yandex-direct – Яндекс директ\
    google_ads – реклама в Google\
    vk_ads – реклама в ВК\
    instagram_ads – реклама в instagram\
    facebook_ads – реклама в facebook\
    referal – акция «приведи друга»\
    Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  \
<>    
 

Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [6]:
# Размер
df.shape

(2747968, 8)

In [7]:
# Пропуски
df.isna().sum()

date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

In [8]:
# Обнаружение дубликатов по всем столбцам
df.duplicated().sum()

0

In [9]:
# Уникальных юзеров
unique_users = df.device_id.nunique()
unique_users

190884

<h2>Задача #1</h2>

In [10]:
# MAU февраля: количество уникальных пользователей, которые зашли в приложение в течение месяца

february= df[df['date'].dt.month == 2]          # датасет за февраль
feb_unique_users = february.device_id.nunique()
feb_unique_users

75032

In [11]:
# Количество установок в январе:
january= df[df['date'].dt.month == 1]          # датасет за январе
jan_installs = january[january['event'] == 'app_install'].shape[0]
jan_installs

80297

<h2>Задача #2</h2>
Когорты по дню установки приложения и конверсия из установки в покупку в течение 7 дней

In [12]:
# Сортировка таблиц по Установкам и Покупкам
installs = df.query('event == "app_install" ')
purchases = df.query('event == "purchase" ')

In [13]:
# кол-во уникальных установок по девайсам
installs.device_id.nunique()

154597

In [14]:
# кол-во уникальных дней
installs.date.nunique()

91

In [15]:
# Таблица с датами первой покупки
first_purchases_dates = purchases.groupby('device_id', as_index=False).agg({'date':'min'})

In [16]:
first_purchases_dates

,device_id,date
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07
...,...,...
70870,35321120,2020-03-25
70871,35326839,2020-03-28
70872,35342310,2020-03-27
70873,35367797,2020-03-29


In [17]:
# Объединяем таблицу с установками и Таблицу с датами первой покупки
inst_n_purch = pd.merge(installs, first_purchases_dates, on = 'device_id', suffixes=('_install', '_first_purchase'))

In [18]:
# Вычитаем кол-во дней прошедших с первой установки
inst_n_purch['days_to_1s_purchase'] = (inst_n_purch['date_first_purchase'] - inst_n_purch['date_install']).dt.days
# Фильтруем до 7 дней от первой установки
seven_days = inst_n_purch[(inst_n_purch['days_to_1s_purchase'] >= 0) & (inst_n_purch['days_to_1s_purchase'] <= 7)]

In [19]:
# Считаем Когорты по дню установки приложения и кол-во установок
cohort_inst = installs.groupby('date', as_index=False).agg({'device_id' : 'count'})
cohort_inst

,date,device_id
0,2020-01-01,3579
1,2020-01-02,3144
2,2020-01-03,2402
3,2020-01-04,1831
4,2020-01-05,1671
...,...,...
86,2020-03-27,1199
87,2020-03-28,1091
88,2020-03-29,1117
89,2020-03-30,994


In [20]:
# Считаем Когорты по дню установки приложения и кол-во покуаок в течение 7 дней
cohort_purch = seven_days.groupby('date_install', as_index=False).agg({'device_id' : 'count'}).rename(columns={'date_install':'date'})

In [21]:
cohort_total = pd.merge(cohort_inst, cohort_purch, on = 'date', suffixes=('_install', '_purchase'))

In [22]:
# Conversion rate
cohort_total['conversion_rate_%'] = round(cohort_total['device_id_purchase'] / cohort_total['device_id_install'] * 100, 1)

In [23]:
# Renaming columns
cohort_total = cohort_total.rename(columns={'device_id_install':'installs_total', 'device_id_purchase':'purchases_within_7_days_total'})

In [24]:
cohort_total

,date,installs_total,purchases_within_7_days_total,conversion_rate_%
0,2020-01-01,3579,1408,39.3
1,2020-01-02,3144,1186,37.7
2,2020-01-03,2402,834,34.7
3,2020-01-04,1831,639,34.9
4,2020-01-05,1671,587,35.1
...,...,...,...,...
86,2020-03-27,1199,277,23.1
87,2020-03-28,1091,209,19.2
88,2020-03-29,1117,223,20.0
89,2020-03-30,994,204,20.5


In [25]:
max_conversion_row = cohort_total.loc[cohort_total['conversion_rate_%'].idxmax()]

In [26]:
# Когорта с наибольшей конверсией  
max_conversion_row

date                             2020-01-01 00:00:00
installs_total                                  3579
purchases_within_7_days_total                   1408
conversion_rate_%                               39.3
Name: 0, dtype: object

<h2>Задача #3</h2>
С какого платного маркетингового канала пришло больше всего новых пользователей? 

In [27]:
# Допустим новый пользователь = Первая установка
installs

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
2515061,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads
2515062,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-
2515063,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-
2515064,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-
2515065,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal
...,...,...,...,...,...,...,...,...
2669653,2020-03-31,app_install,NaN,ios,19717968,female,Moscow,-
2669654,2020-03-31,app_install,NaN,android,32966665,female,Saint-Petersburg,-
2669655,2020-03-31,app_install,NaN,ios,6335964,female,Moscow,referal
2669656,2020-03-31,app_install,NaN,android,29155826,male,Saint-Petersburg,vk_ads


In [28]:
# Ответ : Яндекс
installs.groupby('utm_source', as_index=False).agg({'device_id' : 'count'}).sort_values('device_id', ascending=False)

,utm_source,device_id
0,-,32460
6,yandex-direct,29368
2,google_ads,26286
5,vk_ads,23189
3,instagram_ads,20096
1,facebook_ads,13916
4,referal,9282


<h2>Задача #4</h2>
Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

In [29]:
# Воронка
'''
app_install – установка приложения
app_start – открыть приложения
register – зарегистрироваться
search – перейти на страницу поиска товаров (каталог)
open_item – открыть товар
choose_item – отправить товар в корзину
tap_basket – перейти в корзину
purchase – подтверждение покупки
'''

'\napp_install – установка приложения\napp_start – открыть приложения\nregister – зарегистрироваться\nsearch – перейти на страницу поиска товаров (каталог)\nopen_item – открыть товар\nchoose_item – отправить товар в корзину\ntap_basket – перейти в корзину\npurchase – подтверждение покупки\n'

In [30]:
df['register_day'] = ''

In [31]:
df.loc[df['event'] == 'register', 'register_day'] = df.loc[df['event'] == 'register', 'date']

In [32]:
df['register_day'] = pd.to_datetime(df['register_day'])

In [33]:
df['register_day'] = df.groupby('device_id')['register_day'].transform('first')

In [34]:
df['date_after_register'] = df['date'] > df['register_day']

In [35]:
df_filtered = df[df.groupby('device_id')['date_after_register'].transform('any')]

In [36]:
df_event = df_filtered.query('date_after_register!=False')

In [37]:
df_event_new = df_event.groupby('event', as_index=False) \
    .agg({'device_id': 'count'})

In [38]:
df_event_new.sort_values('device_id', ascending=False)

,event,device_id
0,app_start,441486
3,search,417101
1,choose_item,314473
4,tap_basket,217256
2,purchase,73630


In [39]:
# Создаем словарь с порядковыми номерами для каждого события
event_order = {'app_install': 0, 'app_start': 1, 'register': 2, 'search': 3, 'open_item': 4, 'choose_item': 5, 'tap_basket': 6, 'purchase': 7}

# Добавляем новый столбец 'event_order' с порядковыми номерами
df_event_new['event_order'] = df_event_new['event'].map(event_order)

# Сортируем датафрейм по столбцу 'event_order'
df_sorted = df_event_new.sort_values('event_order')

In [40]:
df_sorted
df_sorted['device_id_diff'] = df_sorted['device_id'].diff().fillna(df_sorted['device_id'])


In [41]:
df_sorted

,event,device_id,event_order,device_id_diff
0,app_start,441486,1,441486.0
3,search,417101,3,-24385.0
1,choose_item,314473,5,-102628.0
4,tap_basket,217256,6,-97217.0
2,purchase,73630,7,-143626.0


<h2>Задача #5</h2>
Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

In [42]:
installs_cr = installs.groupby('utm_source', as_index=False).agg({'device_id' : 'nunique'})

In [43]:
purchases_cr = purchases.groupby('utm_source', as_index=False).agg({'device_id' : 'nunique'})

In [44]:
sources_cr = pd.merge(installs_cr, purchases_cr, on = 'utm_source', suffixes=('_install', '_purchase') )

In [45]:
sources_cr['cr'] = round(sources_cr['device_id_purchase'] / sources_cr['device_id_install'] * 100, 1)

In [46]:
sources_cr

,utm_source,device_id_install,device_id_purchase,cr
0,-,32460,16598,51.1
1,facebook_ads,13916,9017,64.8
2,google_ads,26286,11339,43.1
3,instagram_ads,20096,10762,53.6
4,referal,9282,6362,68.5
5,vk_ads,23189,12364,53.3
6,yandex-direct,29368,12028,41.0


<h2>Задача #6</h2>
Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [52]:
purchases = purchases.sort_values('date')

In [57]:
purchases_1 = purchases.drop_duplicates('device_id')

In [58]:
purchases_1

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
7691,2020-01-01,purchase,265.5,android,6081155,female,Moscow,-
7693,2020-01-01,purchase,98.5,ios,7458607,female,Moscow,instagram_ads
7694,2020-01-01,purchase,304.5,ios,3520947,male,Moscow,instagram_ads
7695,2020-01-01,purchase,153.5,android,1170605,male,Moscow,google_ads
7696,2020-01-01,purchase,318.0,android,16780017,female,Moscow,-
...,...,...,...,...,...,...,...,...
2507469,2020-03-31,purchase,6546.5,android,11864116,male,Saint-Petersburg,google_ads
2507471,2020-03-31,purchase,2857.5,ios,26120362,female,Moscow,google_ads
2507006,2020-03-31,purchase,577.5,android,16233094,female,Moscow,instagram_ads
2507446,2020-03-31,purchase,6963.0,android,11153353,female,Saint-Petersburg,-


In [62]:
# Реферальная программа
purchases_1.groupby('utm_source', as_index=False).agg({'purchase_sum' : 'median'}).sort_values('purchase_sum')

,utm_source,purchase_sum
1,facebook_ads,389.0
2,google_ads,390.5
6,yandex-direct,392.5
5,vk_ads,393.0
3,instagram_ads,393.5
4,referal,395.5
0,-,398.5


<h2>Задача #7</h2>

Данные по затратам на рекламу:

- Яндекс – 10 491 707 руб.

- Гугл – 10 534 878 руб.

- Фейсбук – 8 590 498 руб.

- Инстаграм – 8 561626 руб.

- ВК – 9 553 531руб.

- Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.

Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI?


ROMI = (Валовая прибыль - Маркетинговые расходы) / Маркетинговые расходы

In [70]:
# Доходы с покупок по каналам прожвиженмя
romi_df = df.groupby('utm_source', as_index=False).agg({'purchase_sum' : 'sum'}).sort_values('purchase_sum', ascending=False)

In [71]:
merketing_expenses = [0, 9553531, 8561626, 10491707, 10534878, 8590498, 0]

In [72]:
romi_df['merketing_expenses'] = merketing_expenses

In [73]:
romi_df

,utm_source,purchase_sum,merketing_expenses
0,-,21449749.5,0
5,vk_ads,16389652.5,9553531
3,instagram_ads,14546969.0,8561626
6,yandex-direct,13915368.0,10491707
2,google_ads,12868276.0,10534878
1,facebook_ads,12249901.0,8590498
4,referal,8837044.5,0


In [78]:
romi_df['romi'] = round((romi_df['purchase_sum'] - romi_df['merketing_expenses']) / romi_df['purchase_sum'],2)

In [79]:
# ВК
romi_df

,utm_source,purchase_sum,merketing_expenses,romi
0,-,21449749.5,0,1.00
5,vk_ads,16389652.5,9553531,0.42
3,instagram_ads,14546969.0,8561626,0.41
6,yandex-direct,13915368.0,10491707,0.25
2,google_ads,12868276.0,10534878,0.18
1,facebook_ads,12249901.0,8590498,0.30
4,referal,8837044.5,0,1.00
